<a href="https://colab.research.google.com/github/susyimes/pdf2text/blob/main/pdf2textv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 安装所需的库
!pip install pymupdf
!sudo apt install tesseract-ocr
!pip install pytesseract
!sudo apt-get install tesseract-ocr-chi-sim
!sudo apt-get install tesseract-ocr-eng

# import fitz  # PyMuPDF
# from PIL import Image, ImageEnhance, ImageFilter
# import pytesseract
# import io


# pdf_path = '/content/光伏93题.pdf'  # 替换成你的 PDF 文件路径
# doc = fitz.open(pdf_path)

# all_text = ""
# for page_num in range(len(doc)):
#     page = doc.load_page(page_num)
#     pix = page.get_pixmap(dpi=500)  # 提高 DPI
#     img_data = pix.tobytes("png")
#     image = Image.open(io.BytesIO(img_data))

#     # 图像预处理
#     image = image.filter(ImageFilter.MedianFilter())  # 中值滤波去噪
#     image = ImageEnhance.Contrast(image).enhance(3)  # 增强对比度

#     all_text += pytesseract.image_to_string(image, lang='chi_sim+eng', config='--psm 3 --oem 1')

# print(all_text)
# # 将提取的文本保存到与 PDF 同一路径的 TXT 文件
# txt_path = pdf_path.replace('.pdf', '.txt')

# with open(txt_path, 'w', encoding='utf-8') as file:
#     file.write(all_text)
from google.colab import drive
drive.mount('/content/drive')
import os
import fitz  # PyMuPDF
from PIL import Image, ImageEnhance, ImageFilter
import pytesseract
import io
import cv2
import numpy as np
from concurrent.futures import ThreadPoolExecutor

def process_pdf_to_text(pdf_path, txt_path):
    doc = fitz.open(pdf_path)
    all_text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        pix = page.get_pixmap(dpi=400) # best 400
        img_data = pix.tobytes("png")
        image = Image.open(io.BytesIO(img_data))

        # 图像预处理
        image = image.convert('L')  # 转换为灰度图
        image = image.filter(ImageFilter.MedianFilter())  # 中值滤波去噪
        image = ImageEnhance.Contrast(image).enhance(3)  # 增强对比度

        # # 添加二值化处理

        image = np.array(image)  # 将 PIL 图像转换为 NumPy 数组
        _, binary_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        image = Image.fromarray(binary_image)  # 将 NumPy 数组转换回 PIL 图像

        # OCR 处理 best 6
        all_text += pytesseract.image_to_string(image, lang='chi_sim', config='--psm 6 --oem 3')

    with open(txt_path, 'w', encoding='utf-8') as file:
        file.write(all_text)

def process_pdf_directory(directory):
    with ThreadPoolExecutor(max_workers=2) as executor:
        for filename in os.listdir(directory):
            if filename.endswith('.pdf'):
                pdf_path = os.path.join(directory, filename)
                txt_path = pdf_path.replace('.pdf', '.txt')
                executor.submit(process_pdf_to_text, pdf_path, txt_path)

directory = '/content/pdf/05'  # 替换为你的 PDF 文件夹路径
process_pdf_directory(directory)



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr-chi-sim is already the newest version (1:4.00~git30-7274cfa-1.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr-eng is already the newest version (1:4.00~git30-7274cfa-1.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
